In [56]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/randomforestmodelforstocks/randomRegressorModel.pkl
/kaggle/input/optiver-trading-at-the-close/public_timeseries_testing_util.py
/kaggle/input/optiver-trading-at-the-close/train.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv
/kaggle/input/optiver-trading-at-the-close/optiver2023/competition.cpython-310-x86_64-linux-gnu.so
/kaggle/input/optiver-trading-at-the-close/optiver2023/__init__.py
/kaggle/input/xgbandlinearmodelsforstocks/xgbRegressor.pkl
/kaggle/input/xgbandlinearmodelsforstocks/linearRegressionModel.pkl


In [57]:
# Viz libraries
import seaborn as sns
import matplotlib.pyplot as plt

# For training and evaluation
from sklearn.model_selection import GridSearchCV, train_test_split  
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor 
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.svm import SVR
import tensorflow as tf
from tensorflow import keras
import tensorflow_estimator as tf_estimator

In [58]:
path_train = "/kaggle/input/optiver-trading-at-the-close/train.csv"
path_test = "/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv"
pd.options.display.float_format = '{:.2f}'.format

In [59]:
df0 = pd.read_csv(path_train)

In [60]:
df = df0.copy()

# First look

In [61]:
df.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,1.00,13380276.64,NaN,NaN,1.00,60651.50,1.00,8493.03,1.00,-3.03,0,0_0_0
1,1,0,0,166603.91,-1,1.00,1642214.25,NaN,NaN,1.00,3233.04,1.00,20605.09,1.00,-5.52,0,0_0_1
2,2,0,0,302879.87,-1,1.00,1819368.03,NaN,NaN,1.00,37956.00,1.00,18995.00,1.00,-8.39,0,0_0_2
3,3,0,0,11917682.27,-1,1.00,18389745.62,NaN,NaN,1.00,2324.90,1.00,479032.40,1.00,-4.01,0,0_0_3
4,4,0,0,447549.96,-1,1.00,17860614.95,NaN,NaN,1.00,16485.54,1.00,434.10,1.00,-7.35,0,0_0_4


In [62]:
df.dtypes

stock_id                     int64
date_id                      int64
seconds_in_bucket            int64
imbalance_size             float64
imbalance_buy_sell_flag      int64
reference_price            float64
matched_size               float64
far_price                  float64
near_price                 float64
bid_price                  float64
bid_size                   float64
ask_price                  float64
ask_size                   float64
wap                        float64
target                     float64
time_id                      int64
row_id                      object
dtype: object

In [63]:
df.describe(include="all")

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
count,5237980.00,5237980.00,5237980.00,5237760.00,5237980.00,5237760.00,5237760.00,2343638.00,2380800.00,5237760.00,5237980.00,5237760.00,5237980.00,5237760.00,5237892.00,5237980.00,5237980
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5237980
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0_0_0
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
mean,99.29,241.51,270.00,5715293.10,-0.01,1.00,45100245.28,1.00,1.00,1.00,51813.59,1.00,53575.68,1.00,-0.05,13310.05,NaN
std,57.87,138.53,158.75,20515906.30,0.89,0.00,139841291.28,0.72,0.01,0.00,111421.41,0.00,129355.44,0.00,9.45,7619.27,NaN
min,0.00,0.00,0.00,0.00,-1.00,0.94,4316.61,0.00,0.79,0.93,0.00,0.94,0.00,0.94,-385.29,0.00,NaN
25%,49.00,122.00,130.00,84534.15,-1.00,1.00,5279575.17,1.00,1.00,1.00,7374.72,1.00,7823.70,1.00,-4.56,6729.00,NaN
50%,99.00,242.00,270.00,1113604.44,0.00,1.00,12882638.56,1.00,1.00,1.00,21969.00,1.00,23017.92,1.00,-0.06,13345.00,NaN
75%,149.00,361.00,410.00,4190951.00,1.00,1.00,32700130.58,1.00,1.00,1.00,55831.68,1.00,57878.41,1.00,4.41,19907.00,NaN


# Handling Null Values

In [64]:
df.isna().sum()

stock_id                         0
date_id                          0
seconds_in_bucket                0
imbalance_size                 220
imbalance_buy_sell_flag          0
reference_price                220
matched_size                   220
far_price                  2894342
near_price                 2857180
bid_price                      220
bid_size                         0
ask_price                      220
ask_size                         0
wap                            220
target                          88
time_id                          0
row_id                           0
dtype: int64

In [65]:
df.dropna(subset=["imbalance_size", "reference_price","matched_size","ask_price","wap","target"], inplace=True)

In [66]:
df_with_far_near_price = df.dropna(subset=["imbalance_size", "reference_price","matched_size","ask_price","wap","target","far_price", "near_price"])

# Looking at the Spread of the Values

In [67]:
"""
for col in df.columns:
    sns.boxplot(df[col])
    plt.title(f"Plot for {col}")
    plt.show()
    print("-----------------------------------------------------------------------------")
"""

'\nfor col in df.columns:\n    sns.boxplot(df[col])\n    plt.title(f"Plot for {col}")\n    plt.show()\n    print("-----------------------------------------------------------------------------")\n'

# Encoding Categorical Variables

In [68]:
df["stock_id"] = df["stock_id"].astype("object")
df["time_id"] = df["time_id"].astype("object")
df["date_id"] = df["date_id"].astype("object")
df["imbalance_buy_sell_flag"] = df["imbalance_buy_sell_flag"].astype("object")

In [69]:
df = pd.get_dummies(df, columns=["imbalance_buy_sell_flag"])

In [70]:
df.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id,imbalance_buy_sell_flag_-1,imbalance_buy_sell_flag_0,imbalance_buy_sell_flag_1
0,0,0,0,3180602.69,1.00,13380276.64,NaN,NaN,1.00,60651.50,1.00,8493.03,1.00,-3.03,0,0_0_0,False,False,True
1,1,0,0,166603.91,1.00,1642214.25,NaN,NaN,1.00,3233.04,1.00,20605.09,1.00,-5.52,0,0_0_1,True,False,False
2,2,0,0,302879.87,1.00,1819368.03,NaN,NaN,1.00,37956.00,1.00,18995.00,1.00,-8.39,0,0_0_2,True,False,False
3,3,0,0,11917682.27,1.00,18389745.62,NaN,NaN,1.00,2324.90,1.00,479032.40,1.00,-4.01,0,0_0_3,True,False,False
4,4,0,0,447549.96,1.00,17860614.95,NaN,NaN,1.00,16485.54,1.00,434.10,1.00,-7.35,0,0_0_4,True,False,False


In [71]:
df.dtypes

stock_id                       object
date_id                        object
seconds_in_bucket               int64
imbalance_size                float64
reference_price               float64
matched_size                  float64
far_price                     float64
near_price                    float64
bid_price                     float64
bid_size                      float64
ask_price                     float64
ask_size                      float64
wap                           float64
target                        float64
time_id                        object
row_id                         object
imbalance_buy_sell_flag_-1       bool
imbalance_buy_sell_flag_0        bool
imbalance_buy_sell_flag_1        bool
dtype: object

In [72]:
#df_prep = pd.get_dummies(df[:round(len(df)/5)])

In [73]:
#df_prep.columns

# Model Building

## Split the data

In [152]:
X = df.drop(columns=["row_id", "stock_id", "date_id", "time_id", "far_price", "near_price","target"])
y = df["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Build Different Models

In [153]:
models_stats = {}

### RandomForestRegressor

In [76]:
#model = RandomForestRegressor(random_state=42)

In [77]:
#%%time
#model.fit(X_train, y_train)

In [78]:
#with open("/kaggle/input/randomforestmodelforstocks/randomRegressorModel.pkl", "wb") as file:
    #pickle.dump(model, file)

In [131]:
with open("/kaggle/input/randomforestmodelforstocks/randomRegressorModel.pkl", "rb") as file:
    model = pickle.load(file)

In [132]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Add the metrics' values to the stats dictionary
model_metrics = {}
model_metrics["mse"] = mse
model_metrics["r2_score"] = r2

models_stats["RandomForestRegressor"] = model_metrics

In [133]:
zipped = zip(y_pred, y_test)

# Iterate over the first five pairs from the zipped object
for i in range(40):
    predicted, real = next(zipped)
    print(f'Predicted value: {predicted}, real value: {real}, difference between them: {predicted-real}')

Predicted value: 9.239912000000002, real value: 9.239912, difference between them: 1.7763568394002505e-15
Predicted value: 0.28967857000000147, real value: 0.28967857, difference between them: 1.4432899320127035e-15
Predicted value: 7.17043900000009, real value: 7.170439, difference between them: 8.970602038971265e-14
Predicted value: -1.5002489000000017, real value: -1.5002489, difference between them: -1.5543122344752192e-15
Predicted value: 1.3494491999999982, real value: 1.3494492, difference between them: -1.7763568394002505e-15
Predicted value: -2.0700692999999997, real value: -2.0700693, difference between them: 4.440892098500626e-16
Predicted value: 38.22034569999993, real value: 38.21969, difference between them: 0.0006556999999318691
Predicted value: 6.749629999999999, real value: 6.74963, difference between them: -8.881784197001252e-16
Predicted value: -5.4502486999999755, real value: -5.4502487, difference between them: 2.4868995751603507e-14
Predicted value: -1.27971170000

### LinearRegression

In [81]:
#model = LinearRegression()

In [82]:
#%%time
#model.fit(X_train, y_train)

In [83]:
#with open("linearRegressionModel.pkl", "wb") as file:
    #pickle.dump(model, file)

In [84]:
with open("/kaggle/input/xgbandlinearmodelsforstocks/linearRegressionModel.pkl", "rb") as file:
    model = pickle.load(file)

In [85]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Add the metrics' values to the stats dictionary
model_metrics = {}
model_metrics["mse"] = mse
model_metrics["r2_score"] = r2

models_stats["LinearRegression"] = model_metrics

### Gradient Boosting Regressor

In [86]:
#model = xgb.XGBRegressor(n_estimators=100, random_state=42)

In [87]:
#%%time
#model.fit(X_train, y_train)

In [88]:
#with open("xgbRegressor.pkl", "wb") as file:
    #pickle.dump(model, file)

In [120]:
with open("/kaggle/input/xgbandlinearmodelsforstocks/xgbRegressor.pkl", "rb") as file:
    model = pickle.load(file)

In [121]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Add the metrics' values to the stats dictionary
model_metrics = {}
model_metrics["mse"] = mse
model_metrics["r2_score"] = r2

models_stats["XGBRegressor"] = model_metrics

In [125]:
# Iterate over the zipped object
zipped = zip(y_pred, y_test)
for element in zipped:
    predicted, real = element
    print(f'Predicted value: {x_element}, real value: {y_element}, difference between them: {x_element-y_element}')

True

In [130]:
zipped = zip(y_pred, y_test)

# Iterate over the first five pairs from the zipped object
for i in range(40):
    predicted, real = next(zipped)
    print(f'Predicted value: {predicted}, real value: {real}, difference between them: {predicted-real}')

Predicted value: 9.239998817443848, real value: 9.239912, difference between them: 8.681744384730905e-05
Predicted value: 0.2947447896003723, real value: 0.28967857, difference between them: 0.00506621960037229
Predicted value: 7.158048629760742, real value: 7.170439, difference between them: -0.01239037023925782
Predicted value: -1.5065590143203735, real value: -1.5002489, difference between them: -0.006310114320373428
Predicted value: 1.353078007698059, real value: 1.3494492, difference between them: 0.0036288076980590667
Predicted value: -2.115471363067627, real value: -2.0700693, difference between them: -0.04540206306762684
Predicted value: 38.236759185791016, real value: 38.21969, difference between them: 0.017069185791015684
Predicted value: 6.7485551834106445, real value: 6.74963, difference between them: -0.0010748165893552653
Predicted value: -5.443143844604492, real value: -5.4502487, difference between them: 0.007104855395508203
Predicted value: -1.2664828300476074, real va

In [127]:
x = [1,2,3,4,5]
y = [7,8,44,53,521]
zipped = zip(x,y)
# Iterate over the zipped object
for element in zipped:
    x_element, y_element = element
    print(f'x: {x_element}, y: {y_element}')

TypeError: 'zip' object is not subscriptable

# Support Vector Regressor

In [36]:
model = SVR(kernel='rbf', C=1.0)

In [ ]:
%%time
model.fit(X_train, y_train)

In [ ]:
with open("SVRModel.pkl", "wb") as file:
    pickle.dump(model, file)

In [ ]:
\
  

### Neural Newtwork

In [95]:
input_dim = X_train.shape[1]

In [100]:
X_train = X_train.astype("float")

In [101]:
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(input_dim,)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # Output layer
])
model.compile(optimizer='adam', loss='mean_squared_error')

In [103]:
%%time
model.fit(X_train, y_train, epochs=3)

Epoch 1/3
122760/122760 [==============================] - 232s 2ms/step - loss: 135020281856.0000
Epoch 2/3
122760/122760 [==============================] - 232s 2ms/step - loss: 89.4924
Epoch 3/3
122760/122760 [==============================] - 230s 2ms/step - loss: 89.4927
CPU times: user 14min 9s, sys: 1min 11s, total: 15min 20s
Wall time: 12min 23s


In [104]:
# Save the model
tf.keras.models.save_model(model, 'neuralNetworkModel_v1')

In [92]:
# Load the SavedModel
model = tf.saved_model.load('/kaggle/working/neuralNetworkModel.tf')

In [105]:
loaded_model = tf.keras.models.load_model('/kaggle/working/neuralNetworkModel_v1')

In [94]:
model

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x7dd1316240d0>

In [109]:
X_tf_test = X_test.astype("float")

In [110]:
# Make predictions on the test set
y_pred = model.predict(X_tf_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Add the metrics' values to the stats dictionary
model_metrics = {}
model_metrics["mse"] = mse
model_metrics["r2_score"] = r2

models_stats["NeuralSequentialNetwork"] = model_metrics

40920/40920 [==============================] - 53s 1ms/step


In [119]:
np.unique(y_pred)

array([-0.03866053], dtype=float32)

# Comparison of Key Metrics between different Models

In [111]:
# Convert the inner dictionaries to Series
data_series = {key: pd.Series(value) for key, value in models_stats.items()}

In [112]:
# Create the DataFrame
df_comparison = pd.DataFrame(data_series).T

In [113]:
df_comparison  

,mse,r2_score
RandomForestRegressor,0.01,1.00
LinearRegression,0.00,1.00
XGBRegressor,0.00,1.00
NeuralSequentialNetwork,88.95,-0.00


# Making predictions using RandomForestRegressor Model

##### Upload the model

In [136]:
with open("/kaggle/input/randomforestmodelforstocks/randomRegressorModel.pkl", "rb") as file:
    model_rf = pickle.load(file)

##### Prepare testing set

In [137]:
test_df = pd.read_csv("/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv")

In [141]:
prediction_features = test_df[['seconds_in_bucket', 'imbalance_size', 'reference_price',
       'matched_size', 'bid_price', 'bid_size', 'ask_price', 'ask_size', 'wap', "imbalance_buy_sell_flag"]]

In [144]:
prediction_features["imbalance_buy_sell_flag"] = prediction_features["imbalance_buy_sell_flag"].astype("object")

/tmp/ipykernel_32/2170161959.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prediction_features["imbalance_buy_sell_flag"] = prediction_features["imbalance_buy_sell_flag"].astype("object")


In [147]:
pred_fts = pd.get_dummies(prediction_features)  

In [148]:
pred_fts

,seconds_in_bucket,imbalance_size,reference_price,matched_size,bid_price,bid_size,ask_price,ask_size,wap,imbalance_buy_sell_flag_-1,imbalance_buy_sell_flag_0,imbalance_buy_sell_flag_1
0,0,3753451.43,1.00,11548975.43,1.00,22940.00,1.00,9177.60,1.00,True,False,False
1,0,985977.11,1.00,3850033.97,1.00,1967.90,1.00,19692.00,1.00,True,False,False
2,0,599128.74,1.00,4359198.25,1.00,4488.22,1.00,34955.12,1.00,False,False,True
3,0,2872317.54,1.00,27129551.64,1.00,16082.04,1.00,10314.00,1.00,True,False,False
4,0,740059.14,1.00,8880890.78,1.00,19012.35,1.00,7245.60,1.00,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
32995,540,2440722.89,1.00,28280361.74,1.00,32257.04,1.00,319862.40,1.00,True,False,False
32996,540,349510.47,1.00,9187699.11,1.00,205108.40,1.00,93393.07,1.00,True,False,False
32997,540,0.00,1.00,12725436.10,1.00,16790.66,1.00,180038.32,1.00,False,True,False
32998,540,1000898.84,1.00,94773271.05,1.00,125631.72,1.00,669893.00,1.00,False,False,True


In [151]:
X.columns

Index(['seconds_in_bucket', 'imbalance_size', 'reference_price',
       'matched_size', 'bid_price', 'bid_size', 'ask_price', 'ask_size', 'wap',
       'target', 'imbalance_buy_sell_flag_-1', 'imbalance_buy_sell_flag_0',
       'imbalance_buy_sell_flag_1'],
      dtype='object')

# API of Time ID--------------------------------------------------------

In [ ]:
input_paths = [path_train, path_test]
group_id_column = "time_id"


In [ ]:
'''
An unlocked version of the timeseries API intended for testing alternate inputs.
Mirrors the production timeseries API in the crucial respects, but won't be as fast.

ONLY works afer the first three variables in MockAPI.__init__ are populated.
'''

from typing import Sequence, Tuple

import pandas as pd


class MockApi:
    def __init__(self):
        '''
        YOU MUST UPDATE THE FIRST THREE LINES of this method.
        They've been intentionally left in an invalid state.

        Variables to set:
            input_paths: a list of two or more paths to the csv files to be served
            group_id_column: the column that identifies which groups of rows the API should serve.
                A call to iter_test serves all rows of all dataframes with the current group ID value.
            export_group_id_column: if true, the dataframes iter_test serves will include the group_id_column values.
        '''
        self.input_paths: Sequence[str] = input_paths
        self.group_id_column: str = group_id_column
        self.export_group_id_column: bool = True
        # iter_test is only designed to support at least two dataframes, such as test and sample_submission
        assert len(self.input_paths) >= 2

        self._status = 'initialized'
        self.predictions = []

    def iter_test(self) -> Tuple[pd.DataFrame]:
        '''
        Loads all of the dataframes specified in self.input_paths,
        then yields all rows in those dataframes that equal the current self.group_id_column value.
        '''
        if self._status != 'initialized':

            raise Exception('WARNING: the real API can only iterate over `iter_test()` once.')

        dataframes = []
        for pth in self.input_paths:
            dataframes.append(pd.read_csv(pth, low_memory=False))
        group_order = dataframes[0][self.group_id_column].drop_duplicates().tolist()
        dataframes = [df.set_index(self.group_id_column) for df in dataframes]

        for group_id in group_order:
            self._status = 'prediction_needed'
            current_data = []
            for df in dataframes:
                cur_df = df.loc[group_id].copy()
                # returning single line dataframes from df.loc requires special handling
                if not isinstance(cur_df, pd.DataFrame):
                    cur_df = pd.DataFrame({a: b for a, b in zip(cur_df.index.values, cur_df.values)}, index=[group_id])
                    cur_df.index.name = self.group_id_column
                cur_df = cur_df.reset_index(drop=not(self.export_group_id_column))
                current_data.append(cur_df)
            yield tuple(current_data)

            while self._status != 'prediction_received':
                print('You must call `predict()` successfully before you can continue with `iter_test()`', flush=True)
                yield None

        with open('submission.csv', 'w') as f_open:
            pd.concat(self.predictions).to_csv(f_open, index=False)
        self._status = 'finished'

    def predict(self, user_predictions: pd.DataFrame):
        '''
        Accepts and stores the user's predictions and unlocks iter_test once that is done
        '''
        if self._status == 'finished':
            raise Exception('You have already made predictions for the full test set.')
        if self._status != 'prediction_needed':
            raise Exception('You must get the next test sample from `iter_test()` first.')
        if not isinstance(user_predictions, pd.DataFrame):
            raise Exception('You must provide a DataFrame.')

        self.predictions.append(user_predictions)
        self._status = 'prediction_received'


def make_env():
    return MockApi()

In [ ]:
mock_api = make_env()

In [ ]:
df0.iter_test()

In [ ]:
for dataframes_tuple in mock_api.iter_test():
    # Perform operations on the data
    if dataframes_tuple:
        data_from_test, data_from_submission = dataframes_tuple
        # Your processing logic here
        print(data_from_test)
        print(data_from_submission)
        print("Next Iteration")
        # You can access the dataframes in dataframes_tuple
        # Example: data_from_test, data_from_submission
